In [8]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
df = pd.read_csv("chat_exported.txt", header=None, encoding='utf-8', delimiter ='qwerty')#,on_bad_lines='skip')
df.columns = ['base']

#Establecer el idioma en el que estaba configurado wsp
language_settings = "ingles"

#Cantidad de mensajes
df.shape

(40035, 1)

In [9]:
#Extracción de columnas base
df["fecha"] = df["base"].str.split(",", expand=True)[0]
df["time"] = df["base"].str.split(" ", expand=True)[1]
df["remitente"] = df["base"].str.split("-", expand=True)[1].str.split(":", expand=True)[0]
df["mensaje"] = df["base"].str.split(":", expand=True)[2]


In [10]:
#Extracción de columnas adicionales dependiendo de la configuración
if language_settings == "español":
    df["part_day"] = df["time"].str.slice(-5)[1]
    df["minuto"] = df["time"].str.split(":", expand=True)[1].str.slice(0,2)
    df["hora"] = df["time"].str.split(":", expand=True)[0]
else:
    df["part_day"] = df["base"].str.split(" ", expand=True)[4]
    df["hora"] = df["time"].str.split(":", expand=True)[0]
    df["minuto"] = df["time"].str.split(":", expand=True)[1]



In [11]:
#Proposed KPIs
# Most repeated words 
# Messages by hour
# Messages by date
# Day with more messages
# Total number of messages
# Most used emojis
# number of pictores
# One Image
# Number of meesages by sender
# Filter by month, sender and hora
